In [169]:
# to include packages
import pandas as pd
import numpy as np
from datetime import datetime

In [46]:
eua_daily = pd.read_csv('original_data/EUA_daily.csv')
eua_daily.dropna(inplace=True)
eua_daily['volume'] = eua_daily['volume'].astype('str').apply(lambda x: float(x[:-1])*1000)
eua_daily['Market_size'] = eua_daily['volume']*eua_daily['price']

In [48]:
eua_daily[['date','price','Market_size']]
eua_daily.to_csv('daily_eua.csv',index=False)

In [49]:
df = pd.read_csv('original_data/Concentration_daily.csv')
lst_of_col = ['IR', 'Oil', 'Coal',  'NG', 'USEU',   'S&P_clean', 'DAX',  'GDP', 'Power',  ]

In [69]:
df_dict = {}
for col in lst_of_col:
    col_num = np.where(df.columns == col)[0][0]
    col_date_num = col_num - 1
    df_temp = df.iloc[:, col_date_num:col_num+1]
    df_temp.dropna(inplace=True)
    
    if col == 'IR':
        df_temp.rename(columns = {df_temp.columns[0]:'date', df_temp.columns[1]:'value'}, inplace=True)
        df_temp.to_csv(f'processed_data/daily_all_{col}.csv', index = False)
    elif col in ['Oil', 'Coal',  'NG', 'USEU',   'S&P_clean', 'DAX',]:
        df_temp.rename(columns = {df_temp.columns[0]:'date', df_temp.columns[1]:'value'}, inplace=True)
        df_temp.to_csv(f'processed_data/daily_{col}.csv', index = False)
    elif col in ['GDP', 'Power']:
        df_temp.rename(columns = {df_temp.columns[0]:'quarter', df_temp.columns[1]:'value'}, inplace=True)
        df_temp.to_csv(f'processed_data/quarterly_{col}.csv', index = False)
    df_dict[col] = df_temp

In [72]:
df_dict['EUA'] = eua_daily[['date','price']]
df_dict['Market_size'] = eua_daily[['date','Market_size']]

In [74]:
# import pickle

# # Save to a pickle file
# with open('processed_data/concatenate_data.pkl', 'wb') as pickle_file:
#     pickle.dump(df_dict, pickle_file)

## TODO: this is how to call the saved dictionary
# with open('processed_data/concatenate_data.pkl', 'rb') as pickle_file:
#     df_dict = pickle.load(pickle_file)

# print(loaded_data)

In [151]:
lst_of_col = ['IR', 'Oil', 'Coal',  'NG', 'USEU',   'S&P_clean', 'DAX',  'GDP', 'Power', 'EUA', 'Market_size']
df_all = df_dict['IR'].rename(columns = {'date': 'date', 'value': 'IR'})

for col in ['Oil', 'Coal',  'NG', 'USEU',   'S&P_clean', 'DAX', ]:
    df_temp = df_dict[col].rename(columns = {'date': 'date', 'value': col})
    df_all = pd.merge(left = df_all, right = df_temp, on = 'date', how = 'left')
    
df_temp = df_dict['EUA'].rename(columns = {'date': 'date', 'price': 'EUA'})
df_all = pd.merge(left = df_all, right = df_temp, on = 'date', how = 'left')
df_temp = df_dict['Market_size']
df_all = pd.merge(left = df_all, right = df_temp, on = 'date', how = 'left')

In [ ]:
df_all.head()

In [153]:
quarter_mapping = {
    'Q1': '1',  # January
    'Q2': '4',  # April
    'Q3': '7',  # July
    'Q4': '10'   # October
}
def convert_quarter(quarter):
    q, year = quarter.split()
    month = quarter_mapping[q]
    return f"{month}/1/{year}" #10/7/2024

In [154]:
df_temp = df_dict['Power'].copy()
df_temp['quarter'] = df_temp['quarter'].apply(convert_quarter)
df_temp = df_temp.rename(columns = {'quarter':'date', 'value':'Power'})
df_all = pd.merge(left = df_all, right = df_temp, on='date', how = 'left')

In [158]:
df_temp = df_dict['GDP'].copy()
df_temp = df_temp.rename(columns = {'quarter':'date', 'value':'GDP'})
df_all = pd.merge(left = df_all, right = df_temp, on='date', how = 'left')

In [ ]:
df_all[['Power', 'GDP']] = df_all[['Power', 'GDP']].fillna(method = 'bfill')


In [179]:
from datetime import datetime
first_day = datetime.strptime( '1/12/2012', '%m/%d/%Y')

In [ ]:
df_all['Date'] = pd.to_datetime(df_all['date'], format = r'%m/%d/%Y')
df_all.tail(15)

In [183]:
df_all_cropped = df_all[df_all['Date']>=first_day].copy()

In [ ]:
df_all_cropped.fillna(method='bfill', inplace = True)
df_all_cropped

In [186]:
df_all_cropped.to_csv('concatenate_data.csv', index=False)